In [44]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
pip install tensorboard

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [47]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:
import tensorflow as tf
import datetime

In [54]:
import tensorflow as tf
from tensorflow.keras import layers
import os

# Prepare a dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:].reshape(60000, 784).astype('float32') / 255
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# Instantiate a simple classification model
model = tf.keras.Sequential([
  layers.Dense(256, activation=tf.nn.relu),
  layers.Dense(256, activation=tf.nn.relu),
  layers.Dense(10)
])

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Instantiate an accuracy metric.
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()

# Compile the model.
model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])

# Prompt the user for an existing log directory.
log_dir = input("Enter an existing log directory path: ")
if not os.path.exists(log_dir):
    print(f"The specified directory '{log_dir}' does not exist.")
    exit()

# Create a TensorBoard callback.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Iterate over the batches of the dataset.
for step, (x, y) in enumerate(dataset):
    # Open a GradientTape.
    with tf.GradientTape() as tape:
        # Forward pass.
        logits = model(x)
        # Loss value for this batch.
        loss_value = loss(y, logits)
    
    # Get gradients of loss wrt the weights.
    gradients = tape.gradient(loss_value, model.trainable_weights)
    
    # Update the weights of our linear layer.
    optimizer.apply_gradients(zip(gradients, model.trainable_weights))

    # Update the running accuracy.
    accuracy.update_state(y, logits)

    # Logging.
    if step % 100 == 0:
        print('Step:', step)
        print('Loss from last step: %.3f' % loss_value)
        print('Total running accuracy so far: %.3f' % accuracy.result())

# Add the TensorBoard callback to the training history.
callbacks = [tensorboard_callback]

# Train the model with the callbacks for TensorBoard.
model.fit(dataset, epochs=5, callbacks=callbacks)


Step: 0
Loss from last step: 2.325
Total running accuracy so far: 0.109
Step: 100
Loss from last step: 0.262
Total running accuracy so far: 0.834
Step: 200
Loss from last step: 0.171
Total running accuracy so far: 0.875
Step: 300
Loss from last step: 0.191
Total running accuracy so far: 0.893
Step: 400
Loss from last step: 0.176
Total running accuracy so far: 0.907
Step: 500
Loss from last step: 0.082
Total running accuracy so far: 0.914
Step: 600
Loss from last step: 0.220
Total running accuracy so far: 0.921
Step: 700
Loss from last step: 0.135
Total running accuracy so far: 0.926
Step: 800
Loss from last step: 0.281
Total running accuracy so far: 0.930
Step: 900
Loss from last step: 0.077
Total running accuracy so far: 0.933


FailedPreconditionError: {{function_node __wrapped__CreateSummaryFileWriter_device_/job:localhost/replica:0/task:0/device:CPU:0}} log/fit is not a directory [Op:CreateSummaryFileWriter] name: 

In [56]:
%tensorboard --logdir log/fit/

Reusing TensorBoard on port 6006 (pid 10564), started 0:00:05 ago. (Use '!kill 10564' to kill it.)

In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import datetime

# Define a função do modelo MLP esparso.
class SparseMLP(tf.keras.Model):
    def __init__(self, num_classes):
        super(SparseMLP, self).__init__()
        self.flatten = Flatten()
        self.fc1 = Dense(128, activation='relu')
        self.fc2 = Dense(num_classes)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.fc1(x)
        return self.fc2(x)

# Prepare a dataset.
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
dataset = tf.data.Dataset.from_tensor_slices(
    (x_train.reshape(60000, 784).astype('float32') / 255, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# Create a log directory for TensorBoard.
log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# A new MLP.
mlp = SparseMLP(10)

# Loss and optimizer.
loss_fn = SparseCategoricalCrossentropy(from_logits=True)
optimizer = SGD(learning_rate=0.1)

# Define a metric for tracking accuracy.
train_accuracy = SparseCategoricalAccuracy()

# Training loop.
for step, (x, y) in enumerate(dataset):
    with tf.GradientTape() as tape:
        # Forward pass.
        logits = mlp(x)

        # External loss value for this batch.
        loss = loss_fn(y, logits)

        # Add the losses created during the forward pass.
        loss += sum(mlp.losses)

        # Get gradients of loss wrt the weights.
        gradients = tape.gradient(loss, mlp.trainable_weights)

    # Update the weights of our linear layer.
    optimizer.apply_gradients(zip(gradients, mlp.trainable_weights))

    # Update the accuracy metric.
    train_accuracy.update_state(y, logits)

    # Logging.
    if step % 100 == 0:
        print('Step %d: Loss: %.3f, Accuracy: %.3f' % (step, loss, train_accuracy.result()))

# Save the model for future use if needed.
mlp.save("sparse_mlp_model")

# Start TensorBoard with the following command:
# tensorboard --logdir logs/fit


Step 0: Loss: 2.396, Accuracy: 0.078
Step 100: Loss: 0.453, Accuracy: 0.784
Step 200: Loss: 0.395, Accuracy: 0.831
Step 300: Loss: 0.143, Accuracy: 0.851
Step 400: Loss: 0.256, Accuracy: 0.866
Step 500: Loss: 0.359, Accuracy: 0.874
Step 600: Loss: 0.299, Accuracy: 0.882
Step 700: Loss: 0.170, Accuracy: 0.887
Step 800: Loss: 0.284, Accuracy: 0.891
Step 900: Loss: 0.102, Accuracy: 0.896


FailedPreconditionError: sparse_mlp_model\variables is not a directory

In [ ]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 19264), started 21:25:24 ago. (Use '!kill 19264' to kill it.)

In [ ]:

from tensorflow.keras import layers


# Prepare a dataset.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(buffer_size=1024).batch(64)

# Instantiate a simple classification model
model = tf.keras.Sequential([
  layers.Dense(256, activation=tf.nn.relu),
  layers.Dense(256, activation=tf.nn.relu),
  layers.Dense(10)
])

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Instantiate an accuracy metric.
accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()

# Create a log directory for TensorBoard.
log_dir = "logs1/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Compile the model.
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[accuracy])

# Train the model with TensorBoard callback.
model.fit(dataset, epochs=5, callbacks=[tensorboard_callback])

# Save the model for future use if needed.
model.save("mnist_classification_model")



Epoch 1/5
938/938 [==============================] - 4s 4ms/step - loss: 0.2203 - sparse_categorical_accuracy: 0.9344
Epoch 2/5
938/938 [==============================] - 3s 4ms/step - loss: 0.0879 - sparse_categorical_accuracy: 0.9722
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 0.0562 - sparse_categorical_accuracy: 0.9818
Epoch 4/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0421 - sparse_categorical_accuracy: 0.9868
Epoch 5/5
938/938 [==============================] - 4s 4ms/step - loss: 0.0321 - sparse_categorical_accuracy: 0.9895


FailedPreconditionError: mnist_classification_model\variables is not a directory

In [ ]:
%tensorboard --logdir logs1/fit

Reusing TensorBoard on port 6006 (pid 19264), started 21:33:14 ago. (Use '!kill 19264' to kill it.)